 Our update step consists of the following:  


$$
\sum_{x_t}a_t(x_t)p(x_{t+1}|x_t)p(z_{t+1}|x_{t+1})
$$
Where $a_t(x_t)$ = recursive step; $p(x_{t+1}|x_t)$ = transition step;  $p(z_{t+1}|x_{t+1})$ = observation step

For the observation step, with a single value $z_{t+1}$ this would normally just be an index into our observation model. 

For example if the observation model gives us : $$ P(Z_t=z | X_t = x)  = \begin{bmatrix} 
0.3 & 0.4 & 0.3 \\
0.2 & 0.3 & 0.5 \\
0.3 & 0.4 & 0.3 
\end{bmatrix}
\quad$$at index z,x



So if $z_t = 1, x_t=2$, and  $ P(z_t=1| x_t = 2) = 0.5 $

If instead $z_t$ is a multi-dimensional vector representing the observed probability of each emotional state. We could then represent  $ P(z_t| x_t)  $ as weighted combination of each possible state given by:
$$
\sum_z Z_{t,z}*P(Z_t=z | X_t = x)
$$
which is equivlent to taking the dot product of $ Z_t$ and  $P(Z_t=z | X_t = x)$


For example if our observation returned score of $$z_t = [0.3,0.4,0.3]$$ we would get: 
$$ P(z_t| x_t = 2) = (0.1)(0.3) + (0.8)(0.4) + (0.1)(0.3) =  0.38$$  
This would be a much more accurate representation of the unconfident observation. 

